# X-ray photoelectron spectroscopy (XPS)
For the source code, see [xps](https://workgraph-collections.readthedocs.io/en/latest/qe/module.html#workgraph_collections.ase.espresso.xps.xps_workgraph).


## Visualizing the WorkGraph Builder


In [1]:
from workgraph_collections.qe.xps import xps_workgraph
from aiida import load_profile
load_profile()

task = xps_workgraph.task()
task.to_html()

## Visualizing the WorkGraph

In [2]:
from workgraph_collections.qe.xps import xps_workgraph
from aiida import load_profile
load_profile()

wg = xps_workgraph()
wg.to_html()

## Example: ETFA molecule

### Prepare the inputs and submit the workflow


In [3]:
from ase.io import read
from aiida import load_profile
from aiida.orm import (
    Dict,
    KpointsData,
    StructureData,
    load_code,
    load_group,
    QueryBuilder,
    Group,
)
from workgraph_collections.qe.xps import xps_workgraph

load_profile()

def load_core_hole_pseudos(pseudo_group="pseudo_demo_pbe"):
    """Load the core hole pseudos."""
    pseudo_group = (
        QueryBuilder().append(Group, filters={"label": pseudo_group}).one()[0]
    )
    pseudos = {"core_hole": {
        pseudo.label.split("_")[0]: pseudo for pseudo in pseudo_group.nodes if "gs" not in pseudo.label
    },
    "gipaw": {
        pseudo.label.split("_")[0]: pseudo for pseudo in pseudo_group.nodes if "gs" in pseudo.label
    }}
    return pseudos, pseudo_group.base.extras.get("correction", {})


# create input structure
etfa = read("datas/ETFA.xyz")
structure = StructureData(ase=etfa)
# create the PW node
# code = load_code("qe-7.2-pw@localhost")
code = load_code("qe-7.2-pw@eiger")
parameters = Dict(
    {
        "CONTROL": {
            "calculation": "scf",
        },
        "SYSTEM": {
            "ecutwfc": 50,
            "ecutrho": 600,
            "occupations": "fixed",
        },
    }
)
kpoints = KpointsData()
kpoints.set_kpoints_mesh([1, 1, 1])
# Load the pseudopotential family.
core_hole_pseudos, correction_energies = load_core_hole_pseudos("pseudo_demo_pbe")
correction_energies = {
    key.split("_")[0]: value["core"] for key, value in correction_energies.items()
}
pseudo_family = load_group("SSSP/1.3/PBE/efficiency")
pseudos = pseudo_family.get_pseudos(structure=structure)
#
metadata = {
    "options": {
        "resources": {
            "num_machines": 1,
            "num_mpiprocs_per_machine": 1,
        },
    }
}
metadata_eiger = {
    "options": {
        "custom_scheduler_commands": "#SBATCH --account=mr32",
        "resources": {
            "num_machines": 1,
            "num_mpiprocs_per_machine": 36,
        },
    }
}
# ===============================================================================
wg = xps_workgraph(
    structure=structure,
    code=code,
    # atoms_list=[0, 1, 2, 3],
    element_list=["C"],
    parameters=parameters,
    kpoints=kpoints,
    pseudos=pseudos,
    is_molecule=True,
    core_hole_pseudos=core_hole_pseudos,
    correction_energies=correction_energies,
    metadata=metadata_eiger,
)
wg.name = "QE, XPS, ETFA"
print("core_hole_pseudos", core_hole_pseudos.keys())
# wg.submit()
wg.run()


core_hole_pseudos dict_keys(['core_hole', 'gipaw'])


06/10/2024 07:36:12 AM <756097> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [56762|WorkGraphEngine|continue_workgraph]: Continue workgraph.


Continue workgraph.


06/10/2024 07:36:12 AM <756097> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [56762|WorkGraphEngine|continue_workgraph]: nodes ready to run: marked_structures
06/10/2024 07:36:12 AM <756097> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [56762|WorkGraphEngine|run_nodes]: Run node: marked_structures, type: CALCFUNCTION


------------------------------------------------------------
executor:  <function get_xspectra_structures at 0x7f954675de40>
args:  [<StructureData: uuid: 5c2d1082-273c-4a9a-9a65-977984354fd8 (pk: 56756)>]
kwargs:  {'structure': <StructureData: uuid: 5c2d1082-273c-4a9a-9a65-977984354fd8 (pk: 56756)>}
var_kwargs:  {'absorbing_elements_list': <List: uuid: 6bb0ba0a-68fb-46a8-873f-213913f285a4 (pk: 56757) value: ['C']>, 'is_molecule_input': <Bool: uuid: 60114c81-3fc8-4140-895c-c6eac836bd25 (pk: 56758) value: True>}
node type: calcfunction/workfunction.
link_label:  supercell uuid: 8702053c-59d8-4d45-8648-bba6c7e55679 (unstored)
link_label:  output_parameters uuid: dd789b6d-bf57-42f1-ba62-cd9beb90d56c (unstored)
link_label:  marked_structures__site_0 uuid: f2bde682-dfb9-4f85-84df-8b6fefc41d4c (unstored)
link_label:  marked_structures__site_1 uuid: ded5fe01-ed43-4cd1-ac41-78da08195e25 (unstored)
link_label:  marked_structures__site_2 uuid: 9c11afc3-1865-442d-94c5-c5d8b6bd9af6 (unstored)


...

------------------------------------------------------------
executor:  <class 'aiida_quantumespresso.calculations.pw.PwCalculation'>
args:  [<StructureData: uuid: ded5fe01-ed43-4cd1-ac41-78da08195e25 (pk: 56767)>, <Dict: uuid: 0562307b-080d-4a49-a42b-8df48b846f46 (pk: 56773)>, {'C': <UpfData: uuid: e990ad52-c36d-45ad-8f85-57eeed5d564d (pk: 6472)>, 'F': <UpfData: uuid: c656bc1d-e22e-4fd2-b13f-577a9eface5f (pk: 615)>, 'H': <UpfData: uuid: e21ed402-1c7f-492d-ac2b-7a31f61e891d (pk: 592)>, 'O': <UpfData: uuid: 7ab4de29-5f82-4382-8d39-883fb29c61c7 (pk: 612)>, 'X': <UpfData: uuid: 2ed40ad1-57b5-46d2-b132-d1b6d5a06112 (pk: 6480)>}, <KpointsData: uuid: a333a56a-4e44-47d1-8567-337522f333a8 (pk: 56775)>]
kwargs:  {'metadata': {'options': {'custom_scheduler_commands': '#SBATCH --account=mr32', 'resources': {'num_machines': 1, 'num_mpiprocs_per_machine': 36}}}, 'code': <InstalledCode: Remote code 'qe-7.2-pw' on eiger pk: 56609, uuid: a6efafe4-b2ec-4f72-9c06-ecc412a6ddfd>, 'settings': <Dict: uuid: 

06/10/2024 07:36:19 AM <756097> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [56782|WorkGraphEngine|run_nodes]: Run node: pw_excited_site_2, type: CALCJOB


------------------------------------------------------------
executor:  <class 'aiida_quantumespresso.calculations.pw.PwCalculation'>
args:  [<StructureData: uuid: 9c11afc3-1865-442d-94c5-c5d8b6bd9af6 (pk: 56768)>, <Dict: uuid: 74a3a52c-6592-44f9-8df4-a1a1dc107f93 (pk: 56776)>, {'C': <UpfData: uuid: e990ad52-c36d-45ad-8f85-57eeed5d564d (pk: 6472)>, 'F': <UpfData: uuid: c656bc1d-e22e-4fd2-b13f-577a9eface5f (pk: 615)>, 'H': <UpfData: uuid: e21ed402-1c7f-492d-ac2b-7a31f61e891d (pk: 592)>, 'O': <UpfData: uuid: 7ab4de29-5f82-4382-8d39-883fb29c61c7 (pk: 612)>, 'X': <UpfData: uuid: 2ed40ad1-57b5-46d2-b132-d1b6d5a06112 (pk: 6480)>}, <KpointsData: uuid: cbb4d0d8-6549-4607-bcf1-186c2fa9e341 (pk: 56778)>]
kwargs:  {'metadata': {'options': {'custom_scheduler_commands': '#SBATCH --account=mr32', 'resources': {'num_machines': 1, 'num_mpiprocs_per_machine': 36}}}, 'code': <InstalledCode: Remote code 'qe-7.2-pw' on eiger pk: 56609, uuid: a6efafe4-b2ec-4f72-9c06-ecc412a6ddfd>, 'settings': <Dict: uuid: 

06/10/2024 07:36:20 AM <756097> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [56782|WorkGraphEngine|run_nodes]: Run node: pw_excited_site_3, type: CALCJOB


------------------------------------------------------------
executor:  <class 'aiida_quantumespresso.calculations.pw.PwCalculation'>
args:  [<StructureData: uuid: 50ffdefc-1a14-4a9e-b9a6-aadb3250453f (pk: 56769)>, <Dict: uuid: 89371200-de21-404e-abba-fc0bcf23a151 (pk: 56779)>, {'C': <UpfData: uuid: e990ad52-c36d-45ad-8f85-57eeed5d564d (pk: 6472)>, 'F': <UpfData: uuid: c656bc1d-e22e-4fd2-b13f-577a9eface5f (pk: 615)>, 'H': <UpfData: uuid: e21ed402-1c7f-492d-ac2b-7a31f61e891d (pk: 592)>, 'O': <UpfData: uuid: 7ab4de29-5f82-4382-8d39-883fb29c61c7 (pk: 612)>, 'X': <UpfData: uuid: 2ed40ad1-57b5-46d2-b132-d1b6d5a06112 (pk: 6480)>}, <KpointsData: uuid: 2cf55d29-22b9-477a-9491-734e07b362a1 (pk: 56781)>]
kwargs:  {'metadata': {'options': {'custom_scheduler_commands': '#SBATCH --account=mr32', 'resources': {'num_machines': 1, 'num_mpiprocs_per_machine': 36}}}, 'code': <InstalledCode: Remote code 'qe-7.2-pw' on eiger pk: 56609, uuid: a6efafe4-b2ec-4f72-9c06-ecc412a6ddfd>, 'settings': <Dict: uuid: 